## Setup

In [1]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.34 🚀 Python-3.10.0 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1650 Ti, 4096MiB)
Setup complete ✅ (16 CPUs, 11.4 GB RAM, 438.6/476.1 GB disk)


In [2]:
!pip install supervision

from IPython import display
display.clear_output()

import supervision as sv
print("supervision.__version__:", sv.__version__)

supervision.__version__: 0.19.0


In [3]:
MODEL = "yolov8x.pt"
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names

# class_ids of interest - car, motorcycle, bus and truck
selected_classes = [1, 3]

YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


In [4]:
import os
from tqdm.notebook import tqdm
import numpy as np
import math
import torch
from IPython import display


## Utils

In [5]:
import VARIABLES
import importlib
importlib.reload(VARIABLES)
from VARIABLES import *

In [25]:
import VARIABLES
import importlib
importlib.reload(VARIABLES)
from VARIABLES import *

import utils
from utils import *
importlib.reload(utils)
from utils import *

videos_folder = "probando"
videos_folder_path = os.path.join("full_recordings", videos_folder)


results_path = os.path.join("processing_results", videos_folder)
counts_path = os.path.join(results_path, "counts")
videos_output_path = os.path.join(results_path, "videos_output")


os.makedirs(results_path, exist_ok=True)
os.makedirs(counts_path, exist_ok=True)
os.makedirs(videos_output_path, exist_ok=True)


max_counters = {
  class_: LineZoneMaxCounterHelper(class_, CLASS_NAMES_DICT[class_]) 
  for class_ in selected_classes
}


vih = VideoInfoHandler()

for file_name in tqdm(os.listdir(videos_folder_path), desc=" Videos remaining", position=0):
  file_path = os.path.join(videos_folder_path, file_name)
  vih.re_init(file_path)
  process_video(
    source_path = file_path,
    target_path = os.path.join(videos_output_path, file_name),
    model = model,
    selected_classes = selected_classes,
    callback = callback,
    vih = vih
  )
  
  for class_ in selected_classes:
    max_counters[class_].update_counting(vih.line_zones, file_name)


save_results(max_counters, selected_classes, counts_path)


 Videos remaining:   0%|          | 0/2 [00:00<?, ?it/s]

 Video processing:   0%|          | 0/365 [00:00<?, ?it/s]

 Video processing:   0%|          | 0/365 [00:00<?, ?it/s]